In [1]:
# Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# General imports
import time
import math
import json
import requests
import functools as ft
import scipy.stats as stats

# Data manipulation and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

# XGBoost and machine learning
import xgboost as xgb
from xgboost import XGBClassifier, plot_importance

# Sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate, learning_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, mean_absolute_error, r2_score, median_absolute_error, PrecisionRecallDisplay, make_scorer
from sklearn.linear_model import RidgeCV, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.decomposition import PCA

# Hyperparameter tuning with Skopt
from skopt import BayesSearchCV
from skopt.space import Integer, Real, Categorical

# Statsmodels
from statsmodels.stats.outliers_influence import variance_inflation_factor

#Saving Model
import joblib

#Database
import sqlite3 
from os import path

In [2]:
ridge_cv_log_loaded = joblib.load('ridge_cv_model.pkl')

In [3]:
all_seasons = []

for season in range(2023, 2025):
    summary_url = f"https://api.nhle.com/stats/rest/en/skater/summary?limit=-1&cayenneExp=seasonId={season}{season+1}%20and%20gameTypeId=2"

    try:
        summary_resp = requests.get(summary_url)
        summary_resp.raise_for_status() 
        summary_json =  summary_resp.json()

        if summary_json['data']:
            df_summary = pd.DataFrame(summary_json['data'])
            all_seasons.append(df_summary)
            df_summary['season'] = f"{season}-{season + 1}"
            print(f"Successfully fetched data for season {season}-{season+1}")
        else:
            print(f"No data returned for season {season}-{season + 1}")
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for season {season}-{season + 1}: {e}")

if all_seasons:
    nhl_api_df = pd.concat(all_seasons, ignore_index=True)
    nhl_api_df = nhl_api_df.groupby('playerId').agg({
            'playerId': 'first',
            'skaterFullName': 'first',
            'positionCode': 'first',
            'gamesPlayed': 'sum',
            'goals': 'sum',
            'assists': 'sum',
            'otGoals': 'sum',
            'gameWinningGoals': 'sum',
            'timeOnIcePerGame': 'mean',
            'teamAbbrevs': 'last'
        }).reset_index(drop = True)
    
print(nhl_api_df)

Successfully fetched data for season 2023-2024
Successfully fetched data for season 2024-2025
     playerId      skaterFullName positionCode  gamesPlayed  goals  assists  \
0     8470600          Ryan Suter            D          123      3       21   
1     8470604         Jeff Carter            C           72     11        4   
2     8470610         Zach Parise            L           30      5        5   
3     8470613         Brent Burns            D          121     14       42   
4     8470621         Corey Perry            R           92     18       16   
..        ...                 ...          ...          ...    ...      ...   
990   8484929         Juha Jaaska            L            1      0        0   
991   8484938          Arttu Hyry            R            2      0        1   
992   8484958     Maxim Tsyplakov            R           39      7       12   
993   8484976         Jere Innala            L            6      0        0   
994   8485105  Nikita Prishchepov    

In [4]:
nhl_api_df = nhl_api_df.loc[(nhl_api_df['positionCode'] != 'D') & (nhl_api_df['gamesPlayed'] >= 35)]
nhl_api_df = nhl_api_df.reset_index(drop = True)
nhl_api_df = nhl_api_df.fillna(0)

nhl_api_df.rename(columns = {'gameWinningGoals': 'game_winning_goals'}, inplace = True)
nhl_api_df.rename(columns = {'otGoals': 'ot_goals'}, inplace = True)
nhl_api_df.rename(columns = {'skaterFullName': 'Player'}, inplace = True)
nhl_api_df.rename(columns={'timeOnIcePerGame': 'time_on_ice_per_game'}, inplace=True)
nhl_api_df['regulation_game_winning'] = nhl_api_df['game_winning_goals'] - nhl_api_df['ot_goals']

In [5]:
nhl_api_df['teamAbbrevs'] = nhl_api_df['teamAbbrevs'].apply(lambda x: x.split(',')[0].strip() if ',' in x else x)

In [6]:
def headshot(row):
    headshot_link = 'https://assets.nhle.com/mugs/nhl/20242025/' + str(row['teamAbbrevs']) + '/' + str(row['playerId']) + '.png'
    return headshot_link

nhl_api_df['headshot'] = nhl_api_df.apply(headshot, axis=1)

In [7]:
def logo(row):
    logo_link = 'https://assets.nhle.com/logos/nhl/svg/' + str(row['teamAbbrevs']) + '_dark.svg'
    return logo_link

nhl_api_df['logo'] = nhl_api_df.apply(logo, axis=1)

In [8]:
start_season = "20232024"
end_season = "20242025"
goals_up_one_url = f"https://www.naturalstattrick.com/playerteams.php?fromseason={start_season}&thruseason={end_season}&stype=2&sit=all&score=u1&stdoi=std&rate=n&team=ALL&pos=F&loc=B&toi=0&gpfilt=none&fd=&td=&tgp=410&lines=single&draftteam=ALL"
goals_down_one_url = f"https://www.naturalstattrick.com/playerteams.php?fromseason={start_season}&thruseason={end_season}&stype=2&sit=all&score=d1&stdoi=std&rate=n&team=ALL&pos=F&loc=B&toi=0&gpfilt=none&fd=&td=&tgp=410&lines=single&draftteam=ALL"
tied_url = f"https://www.naturalstattrick.com/playerteams.php?fromseason={start_season}&thruseason={end_season}&stype=2&sit=all&score=tied&stdoi=std&rate=n&team=ALL&pos=F&loc=B&toi=0&gpfilt=none&fd=&td=&tgp=410&lines=single&draftteam=ALL"
total_url = f"https://www.naturalstattrick.com/playerteams.php?fromseason={start_season}&thruseason={end_season}&stype=2&sit=all&score=all&stdoi=std&rate=n&team=ALL&pos=F&loc=B&toi=0&gpfilt=none&fd=&td=&tgp=410&lines=single&draftteam=ALL"

In [9]:
urls = {
    "goals_up_one": (goals_up_one_url, 'goals_up_by_one'),
    "goals_down_one": (goals_down_one_url, 'goals_down_by_one'),
    "tied": (tied_url, 'goals_when_tied'),
    "total": (total_url, 'total_goals'),
}

dataframes = {}

for name, (url, new_column_name) in urls.items():
    df = pd.read_html(url, header=0, index_col=0, na_values=["-"])[0]
    df.rename(columns={'Goals': new_column_name}, inplace=True)
    dataframes[name] = df

goals_up_one_df = dataframes["goals_up_one"]
goals_down_one_df = dataframes["goals_down_one"]
goals_tied_df = dataframes["tied"]
total_df = dataframes["total"]

In [10]:
goals_up_one_df = goals_up_one_df[['Player', 'GP', 'goals_up_by_one']]
goals_down_one_df = goals_down_one_df[['Player', 'goals_down_by_one']]
goals_tied_df = goals_tied_df[['Player', 'goals_when_tied']]
total_df = total_df[['Player', 'total_goals', 'Shots', 'ixG', 'iFF', 'iSCF', 'iHDCF', 'Rebounds Created', 'iCF']]

dfs_natural_stat = [goals_up_one_df, goals_down_one_df, goals_tied_df, total_df]

merged_natural_stat = ft.reduce(lambda left, right: pd.merge(left, right, on='Player'), dfs_natural_stat)
merged_natural_stat = merged_natural_stat.loc[merged_natural_stat['GP'] >= 35]
merged_natural_stat.rename(columns={'Shots': 'shots'}, inplace=True)
merged_natural_stat.rename(columns={'Rebounds Created': 'rebounds_created'}, inplace=True)

In [11]:
natural_stat_names = ["Pat Maroon", "Alex Kerfoot", "Nicholas Paul", "Zach Sanford", "Alex Wennberg", "Mitchell Marner", "Zach Aston-Reese",  "Max Comtois", "Alexei Toropchenko", "Cameron Atkinson", "Alexander Nylander", "Jacob Lucchini", ] 
nhl_names = ["Patrick Maroon", "Alexander Kerfoot", "Nick Paul", "Zachary Sanford", "Alexander Wennberg", "Mitch Marner", "Zachary Aston-Reese",  "Maxime Comtois", "Alexey Toropchenko", "Cam Atkinson", "Alex Nylander", "Jake Lucchini"]
merged_natural_stat = merged_natural_stat.replace(natural_stat_names, nhl_names)

In [12]:
merged_clutch_goals_prediction = nhl_api_df.merge(merged_natural_stat, on = 'Player', how = 'left')

In [13]:
merged_clutch_goals_prediction.drop(columns = 'GP', axis = 1, inplace = True)

In [14]:
columns = ['ot_goals', 'regulation_game_winning', 'assists', 'goals_up_by_one', 'goals_down_by_one', 'goals_when_tied', 'shots', 'ixG', 'iFF', 'iSCF', 'iHDCF', 'iCF', 'rebounds_created']
for column in columns:
    per_game_string = f"{column}_per_game"
    merged_clutch_goals_prediction[per_game_string] = merged_clutch_goals_prediction[column] / merged_clutch_goals_prediction['gamesPlayed']

In [15]:
merged_clutch_goals_prediction['clutch_score'] = (
    0.425 * merged_clutch_goals_prediction['goals_when_tied_per_game'] + 
    0.425 * merged_clutch_goals_prediction['goals_down_by_one_per_game'] + 
    0.15 * merged_clutch_goals_prediction['ot_goals_per_game']
)

In [16]:
merged_clutch_goals_prediction['clutch_score'] *= 100
merged_clutch_goals_prediction.fillna(0, inplace = True)
merged_clutch_goals_prediction['clutch_score_rank']  = merged_clutch_goals_prediction['clutch_score'].rank(ascending = False, method = 'min')
merged_clutch_goals_prediction['clutch_score'] = merged_clutch_goals_prediction['clutch_score'].apply(lambda x: round(x, 2))
merged_clutch_goals_prediction.sort_values('clutch_score_rank', inplace = True)
merged_clutch_goals_prediction[['Player','clutch_score', 'clutch_score_rank']].head(20)

,Player,clutch_score,clutch_score_rank
260,Auston Matthews,18.21,1.0
177,Leon Draisaitl,15.76,2.0
248,Kirill Kaprizov,15.23,3.0
176,Sam Reinhart,14.65,4.0
243,Artemi Panarin,12.73,5.0
197,Brayden Point,12.41,6.0
183,Dylan Larkin,12.20,7.0
190,David Pastrnak,11.99,8.0
163,Valeri Nichushkin,11.73,9.0
213,Kyle Connor,11.39,10.0


In [17]:
null_rows = merged_clutch_goals_prediction[merged_clutch_goals_prediction.isnull().any(axis=1)]
print("Rows with null values:")
print(null_rows)

Rows with null values:
Empty DataFrame
Columns: [playerId, Player, positionCode, gamesPlayed, goals, assists, ot_goals, game_winning_goals, time_on_ice_per_game, teamAbbrevs, regulation_game_winning, headshot, logo, goals_up_by_one, goals_down_by_one, goals_when_tied, total_goals, shots, ixG, iFF, iSCF, iHDCF, rebounds_created, iCF, ot_goals_per_game, regulation_game_winning_per_game, assists_per_game, goals_up_by_one_per_game, goals_down_by_one_per_game, goals_when_tied_per_game, shots_per_game, ixG_per_game, iFF_per_game, iSCF_per_game, iHDCF_per_game, iCF_per_game, rebounds_created_per_game, clutch_score, clutch_score_rank]
Index: []

[0 rows x 39 columns]


In [18]:
x_var = ['shots_per_game', 'ixG_per_game', 'iFF_per_game', 'iSCF_per_game', 'iHDCF_per_game', 
         'assists_per_game', 'iCF_per_game', 'rebounds_created_per_game', 'time_on_ice_per_game']
X_adjusted = merged_clutch_goals_prediction[x_var]
y_var = 'clutch_score'
y = merged_clutch_goals_prediction[y_var]

In [19]:
X_scaled = StandardScaler().fit_transform(X_adjusted)
X_scaled = np.nan_to_num(X_scaled, nan=0)

epsilon = np.abs(X_scaled.min()) + 1

X_shifted = X_scaled + epsilon

y_log = np.log(y + 1)

X_log = np.log(X_shifted)

y_pred = ridge_cv_log_loaded.predict(X_log)

In [20]:
merged_clutch_goals_prediction['log_adjusted'] = np.log(merged_clutch_goals_prediction['clutch_score'] + 1) * 10
merged_clutch_goals_prediction['predicted_clutch_score_adjusted'] = y_pred * 10
merged_clutch_goals_prediction = merged_clutch_goals_prediction.sort_values(by='predicted_clutch_score_adjusted', ascending = False)
merged_clutch_goals_prediction['log_adjusted'] = merged_clutch_goals_prediction['log_adjusted'].apply(lambda x: round(x, 2))
merged_clutch_goals_prediction['predicted_clutch_score_adjusted'] = merged_clutch_goals_prediction['predicted_clutch_score_adjusted'].apply(lambda x: round(x, 2))

In [21]:
threshold_franchise = merged_clutch_goals_prediction['log_adjusted'].quantile(0.90)
threshold_elite = merged_clutch_goals_prediction['log_adjusted'].quantile(0.85)
threshold_above = merged_clutch_goals_prediction['log_adjusted'].quantile(0.7)
threshold_average = merged_clutch_goals_prediction['log_adjusted'].quantile(0.5)

print(threshold_above)

def create_clutch_rankings(df):

    def assign_tier(score):
        if score >= 23:
            return 'Elite'
        elif score >= 21:
            return 'Top'
        elif score >= 16:
            return 'Average'
        else:
            return 'Low'

    rankings = df.copy()
    '''mean_score = rankings['log_adjusted'].mean()
    std_score = rankings['log_adjusted'].std()
    rankings['standard_deviations'] = (rankings['log_adjusted'] - mean_score) / std_score'''
    
    rankings['tier'] = rankings['log_adjusted'].apply(assign_tier)
    rankings['clutch_percentile'] = rankings['log_adjusted'].rank(pct=True)
    
    rankings['vs_predicted'] = ((rankings['log_adjusted'] - rankings['predicted_clutch_score_adjusted']) / rankings['predicted_clutch_score_adjusted'])
            
    '''def get_prediction_reliability(diff_num):
        if diff_num >= 0:
            if  diff_num <= 10:
                return 'Slightly Overperforming'
            elif diff_num <= 20:
                return 'Overperforming'   
            else:
                return 'Heavily Overperforming'
        elif diff_num <= 0:
            if  diff_num >= -10:
                return 'Slightly Underperforming'
            elif diff_num >= -20:
                return 'Underperforming'   
            else:
                return 'Heavily Underperforming' ''' 

    
    #rankings['Prediction Reliability'] = rankings['vs_predicted'].apply(get_prediction_reliability)
    
    player_output = rankings[[
        'Player',
        'teamAbbrevs',
        'headshot',
        'logo',
        'predicted_clutch_score_adjusted',
        'log_adjusted',
        'clutch_percentile',
        'tier',
        'vs_predicted',
        'clutch_score_rank',
        'goals_when_tied_per_game',
        'goals_down_by_one_per_game',
        'goals_up_by_one_per_game',
        'ot_goals_per_game'
    ]].sort_values('log_adjusted', ascending=False)

    player_output = player_output.reset_index(drop=True)
    
    player_output.columns = ['Player', 'Team', 'Headshot', 'Logo', 'Predicted Clutch Score', 'Actual Clutch Score', 'Clutch Percentile', 'Tier', 'Predicted VS Actual', 'Rank', 
        'goals_when_tied_per_game',
        'goals_down_by_one_per_game',
        'goals_up_by_one_per_game',
        'ot_goals_per_game']

    player_output.to_excel('Player Clutch Statistics.xlsx')
    return player_output

19.08


In [45]:
player_output = create_clutch_rankings(merged_clutch_goals_prediction)

In [47]:
DATA_DIR = r'C:\Users\Work\Desktop'

conn = sqlite3.connect(path.join(DATA_DIR, 'hockey-data.sqlite'))

player_output.to_sql('player_output', conn, index=False, if_exists='replace')

462